In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = LANGCHAIN_API_KEY

In [10]:
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnableLambda

In [11]:
llm = ChatOllama(
    model="qwen3:1.7b",  
    temperature=0,
    format="json",         
    base_url="http://localhost:11434"
)
embedding = OllamaEmbeddings(
    model="all-minilm:l6-v2",
    base_url="http://localhost:11434"
)

C:\Users\xgear\AppData\Local\Temp\ipykernel_10832\3790596850.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  llm = ChatOllama(
C:\Users\xgear\AppData\Local\Temp\ipykernel_10832\3790596850.py:7: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings(


# Logical routing

doc: https://python.langchain.com/docs/use_cases/query_analysis/techniques/routing#routing-to-multiple-indexes

In [12]:
prompt = PromptTemplate(
    template="""You are an expert programming question router.

    Classify the following question into one of the data sources below:
    - "python_docs": For questions about Python.
    - "js_docs": For questions about JavaScript.
    - "golang_docs": For questions about Golang.

    OUTPUT REQUIREMENT:
    Return ONLY a JSON object with the key "datasource". Do not provide any extra explanation.
    Example: {{ "datasource": "python_docs" }}

    Question: {question}
    """,
    input_variables=["question"],
)
question_router = prompt | llm | JsonOutputParser()

In [13]:
question = """Why doesn't the following code work:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""

result = question_router.invoke({"question": question})

In [14]:
result

{'datasource': 'python_docs'}

In [15]:
result['datasource']

'python_docs'

In [16]:
def choose_route(result):
    datasource = result.get("datasource", "").lower()
    
    if "python_docs" in datasource:
        return "chain for python_docs"
    elif "js_docs" in datasource:
        return "chain for js_docs"
    elif "golang_docs" in datasource:
        return "chain for golang_docs"
    else:
        return "chain for default"

from langchain_core.runnables import RunnableLambda

full_chain = question_router | RunnableLambda(choose_route)

In [17]:
full_chain.invoke({"question": question})

'chain for python_docs'

# Semantic routing

In [20]:
from langchain_community.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [21]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

prompt_templates = [physics_template, math_template]
prompt_embeddings = embedding.embed_documents(prompt_templates)

In [22]:
# Route question to prompt 
def prompt_router(input):
    # Embed question
    query_embedding = embedding.embed_query(input["query"])
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    # Chosen prompt 
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | llm
    | StrOutputParser()
)

print(chain.invoke("What's a black hole"))

Using PHYSICS
{
  "answer": "A black hole is a region in space where the gravitational pull is so strong that not even light can escape. It forms when a massive star collapses under its own gravity, creating a singularity (a point of infinite density) surrounded by an event horizon, the boundary from which nothing can return. Black holes can vary in size, from the smallest (stellar black holes) to the largest (supermassive black holes at the centers of galaxies). They are invisible directly but their effects—like gravitational lensing or accretion disks—can be observed."
}
